<a href="https://colab.research.google.com/github/kumar045/Assignment-For-Filed/blob/main/memgpt_autogen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyautogen

In [ ]:
!git clone https://github.com/cpacker/MemGPT.git

In [ ]:
cd MemGPT

In [ ]:
!pip install -e .

In [ ]:
export OPENAI_API_BASE=https://finished-injection-sufficient-molecular.trycloudflare.com/

In [ ]:
export BACKEND_TYPE=webui

In [ ]:
import os
import autogen
import openai

from MemGPT.memgpt.autogen.memgpt_agent import create_autogen_memgpt_agent, create_memgpt_autogen_agent_from_config

# This config is for autogen agents that powered by MemGPT
config_list_memgpt = [
    {
        "model": "gpt-4",
    },
]

config_list = [
    {
        "model": "mistral-7b",
        "api_base": "https://loves-reuters-hope-slope.trycloudflare.com/v1",
        "api_key": "NULL",  # this is a placeholder
        "api_type": "open_ai",
    },
]

USE_MEMGPT = True

USE_AUTOGEN_WORKFLOW = True

DEBUG = False

interface_kwargs = {
    "debug": DEBUG,
    "show_inner_thoughts": DEBUG,
    "show_function_outputs": DEBUG,
}

llm_config = {"config_list": config_list, "seed": 42}
llm_config_memgpt = {"config_list": config_list_memgpt, "seed": 42}

# The user agent
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},
    human_input_mode="TERMINATE",
    default_auto_reply="...",
)

# The agent playing the role of the product manager (PM)
pm = autogen.AssistantAgent(
    name="Product_manager",
    system_message="Creative in software product ideas.",
    llm_config=llm_config,
    default_auto_reply="...",
)

if not USE_MEMGPT:
    coder = autogen.AssistantAgent(
        name="Coder",
        llm_config=llm_config,
    )

else:
    if not USE_AUTOGEN_WORKFLOW:
        coder = create_autogen_memgpt_agent(
            "MemGPT_coder",
            persona_description="I am a 10x engineer, trained in Python. I was the first engineer at Uber "
            "(which I make sure to tell everyone I work with).",
            user_description=f"You are participating in a group chat with a user ({user_proxy.name}) "
            f"and a product manager ({pm.name}).",
            model=config_list_memgpt[0]["model"],
            interface_kwargs=interface_kwargs,
        )
    else:
        coder = create_memgpt_autogen_agent_from_config(
            "MemGPT_coder",
            llm_config=llm_config_memgpt,
            system_message=f"I am a 10x engineer, trained in Python. I was the first engineer at Uber "
            f"(which I make sure to tell everyone I work with).\n"
            f"You are participating in a group chat with a user ({user_proxy.name}) "
            f"and a product manager ({pm.name}).",
            interface_kwargs=interface_kwargs,
        )

groupchat = autogen.GroupChat(agents=[user_proxy, pm, coder], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

user_proxy.initiate_chat(
    manager,
    message="I want to design an app to make me one million dollars in one month. " "Yes, your heard that right.",
)